In [ ]:
import pandas as pd
from textblob import TextBlob
import requests

In [ ]:
df_original = pd.read_json(r"silver\airlines.json")
df_original

In [ ]:
df_flights = pd.read_json(r"gold\flights.json")
df_flights

In [ ]:
airlines_to_keep = df_flights["airline"].unique()


In [ ]:
df_copy = df_original.copy()
df_copy = df_original[df_original["code"].isin(airlines_to_keep)]
df_copy

In [ ]:
with open("api_news.txt") as api:
    api_key = api.readline()

In [ ]:
def get_sentiment(airline):
    if "air" not in airline["name"].lower():
        query = f"{airline['name']} Airline"
    else:
        query = airline['name']

    news_url = f'https://newsapi.org/v2/everything?q={query}&apiKey={api_key}'

    try:
        news_response = requests.get(news_url)
        
        if news_response.ok:
            news_data = news_response.json().get('articles', [])

            if not news_data:
                return "No articles found for the given query."

            score = 0

            for i, article in enumerate(news_data):
                text_to_analyze = article["title"] if article["description"] is None else article["description"]
                score += TextBlob(text_to_analyze).sentiment.polarity

            score /= len(news_data)

            if score > 0:
                sentiment = "positive"
            elif score < -0.05:
                sentiment = "negative"
            else:
                sentiment = "medium"

            return sentiment

        else:
            return "Couldn't parse sentiment. News API response not OK."

    except requests.RequestException as e:
        return f"Error during API request: {e}"

In [ ]:
df_copy["airline_sentiment"] = df_copy.apply(get_sentiment, axis=1)
df_copy

In [ ]:
df_copy.to_json(r"gold\airlines.json", orient="records", indent=2)